<a href="https://colab.research.google.com/github/Gibur13/Digital-Architecture-for-Distance-Based-Sorting/blob/main/a0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Git and Drive Integration

In [35]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import time

### Mount google drive

In [36]:
# Mount google drive to use a persistent directory structure
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Clone the assignment repository

In [ ]:
# Create a directory for the course
%mkdir /content/gdrive/MyDrive/ece5545
%cd /content/gdrive/MyDrive/ece5545

# Create files to hold your Github token and username.
!echo 'YOUR_TOKEN' > /content/gdrive/MyDrive/ece5545/token.txt
!echo 'YOUR_HANDLE' > /content/gdrive/MyDrive/ece5545/git_username.txt

# TODO: Navigate to the created files and replace the contents with your token and username.
# Do NOT place your token inside of the notebook. This way there is no risk that you will push it to your repo.
# Never share your token with anyone, it is basically your github password!

In [ ]:
with open('/content/gdrive/MyDrive/ece5545/token.txt') as f:
    token = f.readline().strip()
# Use another file to store your github username
with open('/content/gdrive/MyDrive/ece5545/git_username.txt') as f:
    handle = f.readline().strip()

YOUR_TOKEN = token
YOUR_HANDLE = handle

!git clone https://{YOUR_TOKEN}@github.com/ML-HW-SYS/a0-{YOUR_HANDLE}.git a0

### Add `src` directory to the Python import path

In [60]:
# import functions from the src directory in repo
import sys
import os
sys.path.insert(0, '/content/gdrive/MyDrive/ece5545/a0/src')
print(sys.path)


['/content/gdrive/MyDrive/ece5545/a0/src', '/content/gdrive/MyDrive/ece5545/a0/src', '/content/gdrive/MyDrive/ece5545/a0/src', '/content/gdrive/MyDrive/ece5545/a0/src', '/content/gdrive/MyDrive/ece5545/a0/src', '/content/gdrive/MyDrive/ece5545/a0/src', '/content/gdrive/MyDrive/ece5545/a0/src', '/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', '/tmp/tmpb38jkkv_', '/content/gdrive/MyDrive/ece5545/a0', '/content/gdrive/MyDrive/ece5545/a0/src', '/content/gdrive/MyDrive/ece5545/a0/src', '/content/gdrive/MyDrive/ece5545/a0/src', '/content/gdrive/MyDrive/ece5545/a0/src/src', '/content/gdrive/MyDrive/ece5545/a0/src']


### Import functions from `src/train_utils.py`

In [62]:
import train_utils
from model import *

SyntaxError: invalid syntax (<ipython-input-62-a10777e85ef3>, line 1)

### Make changes to `src` files and push to repo

In [42]:
# You will likely need to set your git user identification settings
!git config --global user.email "ys2335@cornell.edu"  # update with your email
!git config --global user.name "Yuval Steimberg"   # update with your name

In [43]:
# To run the same tests that are run on Github Classroom
%cd /content/gdrive/MyDrive/ece5545/a0
!python3 -m pytest

/content/gdrive/MyDrive/ece5545/a0
======================================= test session starts ========================================
platform linux -- Python 3.11.11, pytest-8.3.4, pluggy-1.5.0
rootdir: /content/gdrive/MyDrive/ece5545/a0
plugins: anyio-3.7.1, typeguard-4.4.1
collected 1 item                                                                                   

tests/test_model.py .                                                                        [100%]

======================================== 1 passed in 1.64s =========================================


In [44]:
%cd /content/gdrive/MyDrive/ece5545/a0/src
# Navigate to a0/src/model.py and fix the bug (denoted by TODO)
# in the file then commit the changes below
!git commit -am "fixed bug in model.py"
!git push

/content/gdrive/MyDrive/ece5545/a0/src
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	__pycache__/
	../tests/__pycache__/

nothing added to commit but untracked files present (use "git add" to track)
Everything up-to-date


## Training Configuration

In [45]:
batch_size = 128
epochs = 1
lr = 1.0
gamma = 0.7
no_cuda = False
seed = 42
log_interval = 50
save_model = False
dry_run = False

# Derived parameters
use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

## Data Loaders and Optimizer Setup

In [46]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('/content/gdrive/MyDrive/ece5545/data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('/content/gdrive/MyDrive/ece5545/data', train=False,
                       transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

NameError: name 'Net' is not defined

## Training Loop

In [ ]:
import importlib
importlib.reload(train_utils)
# we use importlib to make sure that we are loading
# the latest version of train_utils after any changes
# that you may have done

for epoch in range(1, epochs + 1):
    train_utils.train(model, device, train_loader, optimizer, epoch, log_interval, dry_run)
    train_utils.test(model, device, test_loader)
    scheduler.step()

if save_model:
    torch.save(model.state_dict(), "mnist_cnn.pt")

In [ ]:
# Diagnostic information about the used GPU device
!nvidia-smi